Durante o desafio, optei por usar as tecnologias Python, junto com a biblioteca Selenium para extração de dados da página por meio de seleção HTML (mais especificamente através de Classes e Tags), a biblioteca undetected-chromedriver para que o site não detecte o script e o Jupyter Notebook para documentação do código.
Abaixo, o comando para a instalação de ambas as bibliotecas:

In [ ]:
pip install selenium 
pip install undetected-chromedriver

Declaração das bibliotecas dentro do código:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import time
import csv

Declarando a Função, pedindo os parametros da url, nome da empresa e o maximo de processos.

In [ ]:
def scraping(url, name, max_processos=1500):
    service = Service()
    options = webdriver.ChromeOptions()
    
    partes_env_list = []
    n_processos_list = []
    tribunal_list = []
    localidade_list = []
    uf_list = []
    classe_list = []

Primeiro, criamos a instância do driver do Chrome e configuramos a URL que ele vai acessar. Em seguida, começamos a rolar a tela para garantir que todos os elementos da página sejam capturados. Por fim, verificamos se a quantidade máxima de processos já foi atingida. Isso foi necessário, pois o tempo para carregar toda a página de processos da Nike era muito grande, ocasionando um TimeOut.

In [ ]:
    driver = uc.Chrome(service=service, options=options)

    driver.get(url)

    ultimotamanho = driver.execute_script('return document.body.scrollHeight')

    for contador in range(1000): 
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        # Pega o tamanho total da tela
        novoultimotamanho = driver.execute_script('return document.body.scrollHeight')

        if novoultimotamanho == ultimotamanho:
            break
        # Se não armazena o novo tamanho
        ultimotamanho = novoultimotamanho

        if len(n_processos_list) >= max_processos:
            break

Pegando as partes envolvidas: o driver pega cada parte envolvida e se o começo delas forem diferentes do nome da marca selecionada, adiciona a lista(isso porque no texto existem elementos apenas com os nomes da marca).

In [ ]:
partes = driver.find_elements(By.TAG_NAME, 'Strong')
for i in partes:
        if(name.split()[0] != i.text.split()[0]):
            partes_envolvidas = i.text
            partes_env_list.append(partes_envolvidas)
            if len(partes_env_list) >= max_processos:
                break

Pegamos todos os processos dá página, separamos o N de e deixamos só o número do processo e adicionamos na lista, caso esse valor não exista, adicionamos um vazio.

In [ ]:
    processos = driver.find_elements(By.CLASS_NAME, 'LawsuitCardPersonPage-header-processNumber')
    for i in processos:
        processo_text = i.text.split()
        if len(processo_text) >= 3:
            numero_processos = processo_text[2]
            n_processos_list.append(numero_processos)
        else:
            n_processos_list.append('')
        if len(n_processos_list) >= max_processos:
            break


Dentro da classe 'LawsuitCardPersonPage-body-row-item-text' existiam 3 elementos a ser pego, então separamos eles uns dos outros e adicionamos a respectiva lista.

In [ ]:
texto = driver.find_elements(By.CLASS_NAME, 'LawsuitCardPersonPage-body-row-item-text')
for i in texto:
        if (i.text[:2].isupper()):
            tribunal_localidade_uf = i.text
            uf = tribunal_localidade_uf[-2:]
            uf_list.append(uf)
            tribunal_localidade_uf = tribunal_localidade_uf[:-2]
            tribunal = tribunal_localidade_uf.split()[0]
            tribunal_list.append(tribunal)
            tribunal_localidade_uf = tribunal_localidade_uf[7:]
            localidade = tribunal_localidade_uf.strip(", ")
            localidade_list.append(localidade)
        else:
            classe = i.text
            classe_list.append(classe)

Primeiro calculamos o comprimento máximo entre todas as listas fornecidas e adicionamos elementos vazios ('') à lista até que ela atinja o comprimento necessaria para a formatação ficar correta dentro da planilha. Em seguida colocamos todos os dados em um dicionario e criamos o arquivo csv.

In [ ]:
max_length = max(len(partes_env_list), len(n_processos_list), len(tribunal_list), len(localidade_list), len(uf_list), len(classe_list))

    partes_env_list.extend([''] * (max_length - len(partes_env_list)))
    n_processos_list.extend([''] * (max_length - len(n_processos_list)))
    tribunal_list.extend([''] * (max_length - len(tribunal_list)))
    localidade_list.extend([''] * (max_length - len(localidade_list)))
    uf_list.extend([''] * (max_length - len(uf_list)))
    classe_list.extend([''] * (max_length - len(classe_list)))

    dict_processos = {
        "Tribunal" : tribunal_list,
        "Uf" : uf_list,
        "Localidade" : localidade_list,
        "Procedimento" : classe_list,
        "Partes" : partes_env_list
    }
    
    with open(f'{name}', 'w', newline='') as arquivo:
        dict_writer = csv.DictWriter(arquivo, fieldnames=dict_processos.keys())
        dict_writer.writeheader()
        for i in range(max_length):
            row = {key: dict_processos[key][i] for key in dict_processos}
            dict_writer.writerow(row)

Por fim chamamos a função com a url e nome da empresa.

In [ ]:
scraping("https://www.jusbrasil.com.br/processos/nome/29201840/nike-do-brasil-comercio-e-participacoes-ltda", "Nike do Brasil Comércio e Participações Ltda.csv")

scraping("https://www.jusbrasil.com.br/processos/nome/28654972/adidas-do-brasil-ltda", "Adidas do Brasil Ltda.csv")

scraping("https://www.jusbrasil.com.br/processos/nome/65417283/puma-do-brasil-ltda", "Puma do Brasil Ltda.csv")

scraping("https://www.jusbrasil.com.br/processos/nome/28195165/reebok-produtos-esportivos-brasil-ltda", "Reebok Produtos Esportivos Ltda.csv")

scraping("https://www.jusbrasil.com.br/processos/nome/30873463/asics-brasil-distribuicao-e-comercio-de-artigos-esportivos-ltda", "Asics Brasil, Distribuição e Comércio de Artigos Esportivos Ltda.csv")

scraping("https://www.jusbrasil.com.br/processos/nome/313427154/under-armour-brasil-comercio-e-distribuicao-de-artigos-esportivos-ltda", "Under Armour Brasil Comércio e Distribuição de Artigos Esportivos Ltda.csv")